<a href="https://colab.research.google.com/github/nithecs-biomath/mini-schools/blob/main/Demo_lecture2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUILDING DATA CUBES
## NITheCS mini school: lecture 2

### Install missing packages

In [1]:
%pip install pygbif
%pip install mgrs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 1.8 MB/s eta 0:00:00


### Loading packages

In [2]:
from pygbif import occurrences as occ
import pandas as pd
import geopandas as gpd
from pyproj import Proj, Transformer
from shapely.geometry import mapping
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import io
import zipfile
import mgrs
import math

### Loading Earth Engine

In [3]:
import ee
import eerepr
import geemap

ee.Authenticate(force=True)
ee.Initialize(project='nithecs-436810')

LANDSAT_ID = "LANDSAT/LC08/C02/T1_L2"
BOUNDARIES_ID = 'FAO/GAUL/2015/level1'
WDPA_ID = 'WCMC/WDPA/current/polygons'


dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2021-05-01', '2021-06-01')
sa = ee.FeatureCollection(BOUNDARIES_ID).filter(
    'ADM0_NAME == "South Africa"')

protected_areas = ee.FeatureCollection(WDPA_ID)


sa_landsat = dataset.filterBounds(sa)


### Example of the GBIF API through pygbif

In [4]:
from pygbif import occurrences
data = occurrences.search(taxonKey=212, limit=100)

print(data)

{'offset': 0, 'limit': 100, 'endOfRecords': False, 'count': 2001075924, 'results': [{'key': 4590289036, 'datasetKey': '75085267-1991-4313-9075-6a12e821a51a', 'publishingOrgKey': '1928bdf0-f5d2-11dc-8c12-b8a03c50a862', 'installationKey': 'e44d0fd7-0edf-477f-aa82-50a81836ab46', 'hostingOrganizationKey': '1928bdf0-f5d2-11dc-8c12-b8a03c50a862', 'publishingCountry': 'FR', 'protocol': 'DWC_ARCHIVE', 'lastCrawled': '2024-03-20T17:33:36.256+00:00', 'lastParsed': '2024-03-20T20:37:55.244+00:00', 'crawlId': 1, 'extensions': {}, 'basisOfRecord': 'HUMAN_OBSERVATION', 'occurrenceStatus': 'PRESENT', 'taxonKey': 9362027, 'kingdomKey': 1, 'phylumKey': 44, 'classKey': 212, 'orderKey': 1108, 'familyKey': 2986, 'genusKey': 2498314, 'speciesKey': 9362027, 'acceptedTaxonKey': 9362027, 'scientificName': 'Mareca strepera (Linnaeus, 1758)', 'acceptedScientificName': 'Mareca strepera (Linnaeus, 1758)', 'kingdom': 'Animalia', 'phylum': 'Chordata', 'order': 'Anseriformes', 'family': 'Anatidae', 'genus': 'Mareca'

## GBIF data Cubes

### Generating the Cube

#### Exemplar JSON query for generating a data cube

In [ ]:
{
  "sendNotification": true,
  "notificationAddresses": [
    "maarten.trekels@plantentuinmeise.be"
  ],
  "format": "SQL_TSV_ZIP",
  "sql": "SELECT  PRINTF('%04d-%02d', \"year\", \"month\") AS yearMonth,
   GBIF_EEARGCode(10000, decimalLatitude,  decimalLongitude,  COALESCE(coordinateUncertaintyInMeters, 1000) ) AS eeaCellCode,
   speciesKey,
   species,
   establishmentMeans,
   degreeOfEstablishment,
   pathway,
   COUNT(*) AS occurrences,
   COUNT(DISTINCT recordedBy) AS distinctObservers
   FROM  occurrence
   WHERE occurrenceStatus = 'PRESENT'
   AND countryCode = 'BE'
   AND hasCoordinate = TRUE
   AND NOT ARRAY_CONTAINS(issue, 'ZERO_COORDINATE')
   AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_OUT_OF_RANGE')
   AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_INVALID')
   AND NOT ARRAY_CONTAINS(issue, 'COUNTRY_COORDINATE_MISMATCH')
   AND \"month\" IS NOT NULL
   GROUP BY yearMonth,
   eeaCellCode,
   speciesKey,
   species,
   establishmentMeans,
   degreeOfEstablishment,
   pathway
   ORDER BY  yearMonth DESC,
   eeaCellCode ASC,
   speciesKey ASC"
}


## Loading the Data cube in pandas



#### Download from GitHub

You can download a pre generated data cube from GitHub or any other online resource

In [ ]:
#data = pd.read_csv('https://raw.githubusercontent.com/nithecs-biomath/mini-schools/refs/heads/main/data/sample_data_SA.csv', sep='\t')

#print(data)

         yearmonth\tqdgccode\tfamilykey\tfamily\tspecieskey\tspecies\toccurrences\tfamilycount
0         2024-09\tE016S28AD\t2406\tCrassulaceae\t771688...                                   
1         2024-09\tE016S28BD\t4676\tGeraniaceae\t3826148...                                   
2         2024-09\tE016S28BD\t2406\tCrassulaceae\t733423...                                   
3         2024-09\tE016S28BD\t4259209\tAsphodelaceae\t93...                                   
4         2024-09\tE016S28CB\t6752\tAizoaceae\t8003531\t...                                   
...                                                     ...                                   
14253417  1772-04\tE025S28CC\t7689\tOrchidaceae\t2783834...                                   
14253418  1694-10\tE027S32DB\t2430\tOnagraceae\t3188875\...                                   
14253419   1678-02\tE028S25DC\t7359\tAraneidae\t\t\t1\t9699                                   
14253420  1645-12\tE030S30BB\t7016\tNotodontidae\t

#### Download from Google Drive

In [5]:
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/Shareddrives/NiTheCS mini school/demo_data/Cube_ZA_QDGC_l3.csv', sep='\t')


Mounted at /content/drive


## Getting a Geopackage file from the Grid that you use

In [6]:
# Load QDGC code

input_file = "/content/drive/Shareddrives/NiTheCS mini school/demo_data/qdgc_south_africa.gpkg"

qdgc_ref = gpd.read_file(input_file, layer='tbl_qdgc_03')

In [7]:
print(qdgc_ref)

             qdgc  level_qdgc  cellsize_degrees  lon_center  lat_center  \
0      E016S46CDD           3             0.125     16.4375    -46.9375   
1      E016S46CDB           3             0.125     16.4375    -46.8125   
2      E016S46CBD           3             0.125     16.4375    -46.6875   
3      E016S46CBB           3             0.125     16.4375    -46.5625   
4      E016S46ADD           3             0.125     16.4375    -46.4375   
...           ...         ...               ...         ...         ...   
34422  E037S22DBD           3             0.125     37.9375    -22.6875   
34423  E037S22DBB           3             0.125     37.9375    -22.5625   
34424  E037S22BDD           3             0.125     37.9375    -22.4375   
34425  E037S22BDB           3             0.125     37.9375    -22.3125   
34426  E037S22BBD           3             0.125     37.9375    -22.1875   

         area_km2                                           geometry  
0      132.265148  MULTIPOLY

## Merging the Data cube with the grid

In [8]:
#testing if I can merge data and qdgc

test_merge = pd.merge(data, qdgc_ref, left_on='qdgccode', right_on='qdgc')

print(test_merge)


         yearmonth    qdgccode  familykey        family  specieskey  \
0          2024-09  E016S28ADD     2406.0  Crassulaceae   7716880.0   
1          2024-09  E016S28BCC     2406.0  Crassulaceae   7716880.0   
2          2024-09  E016S28BDD     4676.0   Geraniaceae   3826148.0   
3          2024-09  E016S28BDD     2406.0  Crassulaceae   7334236.0   
4          2024-09  E016S28CBB     6752.0     Aizoaceae   8003531.0   
...            ...         ...        ...           ...         ...   
17829560   1772-04  E025S28CCC     7689.0   Orchidaceae   2783834.0   
17829561   1694-10  E027S32DBB     2430.0    Onagraceae   3188875.0   
17829562   1678-02  E028S25DCB     7359.0     Araneidae         NaN   
17829563   1645-12  E030S30BBC     7016.0  Notodontidae   1824935.0   
17829564   1608-10  E023S33ABC     4334.0        Apidae   5040145.0   

                             species  occurrences  familycount        qdgc  \
0                  Crassula sladenii            1      44434.0  E016S

In [9]:
# Convert to GeoDataFrame

gdf = gpd.GeoDataFrame(test_merge, geometry='geometry')


## Filtering data (e.g. on species)

In [10]:
#check for a single species
filtered_gdf = gdf[gdf['specieskey'].eq(2435350.0)]

print(filtered_gdf)


         yearmonth    qdgccode  familykey        family  specieskey  \
6401       2024-09  E025S33BAD     9427.0  Elephantidae   2435350.0   
6402       2024-09  E025S33BBC     9427.0  Elephantidae   2435350.0   
6403       2024-09  E025S33BCA     9427.0  Elephantidae   2435350.0   
6405       2024-09  E025S33BCB     9427.0  Elephantidae   2435350.0   
6406       2024-09  E025S33BCC     9427.0  Elephantidae   2435350.0   
...            ...         ...        ...           ...         ...   
14708306   1990-01  E025S33BCB     9427.0  Elephantidae   2435350.0   
14708309   1990-01  E025S33BDA     9427.0  Elephantidae   2435350.0   
16222304   1987-01  E031S24BBC     9427.0  Elephantidae   2435350.0   
16427602   1985-09  E031S23ABA     9427.0  Elephantidae   2435350.0   
16427611   1985-09  E031S23ADD     9427.0  Elephantidae   2435350.0   

                     species  occurrences  familycount        qdgc  \
6401      Loxodonta africana           14       5301.0  E025S33BAD   
6402   

## Apply the function to create a list of features

In [11]:

filtered_gdf = filtered_gdf.set_crs(epsg=4326, inplace=False)

data_raw = geemap.geopandas_to_ee(filtered_gdf)

print(type(data_raw))


<class 'ee.featurecollection.FeatureCollection'>


## Visualization of the data cubes on a map with different layers

In [12]:
Map = geemap.Map(layout={"height": "400px", "width": "800px"})


# Add the original data layer in blue
Map.addLayer(data_raw, {"color": "blue"}, "Original data")

Map.addLayer(sa_landsat)

Map.addLayer(protected_areas)


# Set the center of the map to the coordinates
Map.setCenter(-28.50, 29.41)
Map

Map(center=[29.41, -28.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
### Test with NetCDF format

# EBV data cubes in NetCDF format

In [13]:
%pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.1 MB/s eta 0:00:00


In [48]:
import netCDF4 as nc
import xarray as xr


birds_file = xr.open_dataset('/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc')

print(birds_file)

<xarray.Dataset> Size: 26kB
Dimensions:  (lon: 559, lat: 437, time: 1, entity: 486)
Coordinates:
  * lon      (lon) float64 4kB 9.45e+05 9.55e+05 ... 6.515e+06 6.525e+06
  * lat      (lat) float64 3kB 5.305e+06 5.295e+06 ... 9.55e+05 9.45e+05
  * time     (time) datetime64[ns] 8B 2018-01-01
  * entity   (entity) |S37 18kB b'Gavia stellata                       ' ... ...
Data variables:
    crs      |S1 1B ...
Attributes: (12/38)
    Conventions:                CF-1.8, ACDD-1.3, EBV-1.0
    naming_authority:           The German Centre for Integrative Biodiversit...
    history:                    EBV netCDF created using ebvcube, 2024-02-06
    ebv_vocabulary:             https://portal.geobon.org/api/v1/ebv
    ebv_cube_dimensions:        lon, lat, time, entity
    keywords:                   ebv_class: Species populations, ebv_name: Spe...
    ...                         ...
    geospatial_lat_units:       meter
    time_coverage_start:        2013-01-01
    time_coverage_end:       

In [49]:
print(birds_file.variables)

Frozen({'lon': <xarray.IndexVariable 'lon' (lon: 559)> Size: 4kB
array([ 945000.,  955000.,  965000., ..., 6505000., 6515000., 6525000.])
Attributes:
    long_name:      lon
    standard_name:  projection_x_coordinate
    axis:           X
    units:          meter, 'lat': <xarray.IndexVariable 'lat' (lat: 437)> Size: 3kB
array([5305000., 5295000., 5285000., ...,  965000.,  955000.,  945000.])
Attributes:
    long_name:      lat
    standard_name:  projection_y_coordinate
    axis:           Y
    units:          meter, 'time': <xarray.IndexVariable 'time' (time: 1)> Size: 8B
array(['2018-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Attributes:
    long_name:  time
    axis:       T, 'crs': <xarray.Variable ()> Size: 1B
[1 values with dtype=|S1]
Attributes:
    spatial_ref:                     PROJCRS["ETRS89-extended / LAEA Europe",...
    GeoTransform:                    940000 10000 0.0 5310000 0.0 -10000
    grid_mapping_name:               lambert_azimuthal_equal_area
    l

In [50]:
time = birds_file.variables['time']
print(time)

print(birds_file['entity'])

<xarray.IndexVariable 'time' (time: 1)> Size: 8B
array(['2018-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Attributes:
    long_name:  time
    axis:       T
<xarray.DataArray 'entity' (entity: 486)> Size: 18kB
array([b'Gavia stellata                       ',
       b'Gavia arctica                        ',
       b'Tachybaptus ruficollis               ', ...,
       b'Accipiter gentilis all others        ',
       b'Melanitta nigra s. str.              ',
       b'Sylvia subalpina                     '], dtype='|S37')
Coordinates:
  * entity   (entity) |S37 18kB b'Gavia stellata                       ' ... ...
Attributes:
    units:                           1
    ebv_entity_type:                 Species
    ebv_entity_scope:                Bird species listed under the Art. 12 of...
    ebv_entity_classification_name:  Species names as accepted by the Birds D...
    ebv_entity_classification_url:   https://cdr.eionet.europa.eu/help/birds_...
    long_name:                     

In [52]:
# Print a detailed view of all data variables
for var in birds_file.data_vars:
    print(f"Variable: {var}")
    print(birds_file[var])
    print("\n")

Variable: crs
<xarray.DataArray 'crs' ()> Size: 1B
[1 values with dtype=|S1]
Attributes:
    spatial_ref:                     PROJCRS["ETRS89-extended / LAEA Europe",...
    GeoTransform:                    940000 10000 0.0 5310000 0.0 -10000
    grid_mapping_name:               lambert_azimuthal_equal_area
    latitude_of_projection_origin:   52.0
    longitude_of_projection_origin:  10.0
    false_easting:                   4321000.0
    false_northing:                  3210000.0
    semi_major_axis:                 6378137.0
    inverse_flattening:              298.257223563
    longitude_of_prime_meridian:     0.0
    long_name:                       CRS definition




# Random functions to test

In [ ]:
# Function to convert QDGC to lat/long bounding box
def qdgc_to_polygon(qdgc):
    # Parse the longitude and latitude
    lon_deg = int(qdgc[1:4])  # Extract longitude value
    lat_deg = int(qdgc[5:7])  # Extract latitude value

    if qdgc[0] == 'W':  # Western Hemisphere
        lon_deg = -lon_deg
    if qdgc[4] == 'S':  # Southern Hemisphere
        lat_deg = -lat_deg

    # Subdivision (AA, AB, BB, etc.)
    subcell = qdgc[7:]

    # Quarter-degree grid size (0.25° x 0.25°)
    quarter_degree_size = 1

    # Subdivision within quarter-degree cells (1/4 of 0.25° = 0.0625°)
    subcell_size = quarter_degree_size / 4  # Each smaller cell is 0.0625° x 0.0625°

    # Mapping the subcell to the grid position (AA, AB, ..., DD)
    subcell_map = {
        'AA': (0, 0), 'AB': (subcell_size, 0), 'AC': (2 * subcell_size, 0), 'AD': (3 * subcell_size, 0),
        'BA': (0, subcell_size), 'BB': (subcell_size, subcell_size), 'BC': (2 * subcell_size, subcell_size), 'BD': (3 * subcell_size, subcell_size),
        'CA': (0, 2 * subcell_size), 'CB': (subcell_size, 2 * subcell_size), 'CC': (2 * subcell_size, 2 * subcell_size), 'CD': (3 * subcell_size, 2 * subcell_size),
        'DA': (0, 3 * subcell_size), 'DB': (subcell_size, 3 * subcell_size), 'DC': (2 * subcell_size, 3 * subcell_size), 'DD': (3 * subcell_size, 3 * subcell_size)
    }

    lon_shift, lat_shift = subcell_map[subcell]

    # Find the top-left corner of the quarter-degree grid
    lon_min = lon_deg + (0 if qdgc[0] == 'W' else 0.0)
    lat_min = lat_deg + (0 if qdgc[4] == 'S' else 0.0)

    # Shift by the quarter-degree for the QDGC part (quarter-degree grid)
    lon_min += lon_shift
    lat_min += lat_shift

    # Calculate maximum lat and lon
    lat_max = lat_min + subcell_size
    lon_max = lon_min + subcell_size

    # Create the polygon for the grid cell
    return Polygon([(lon_min, lat_min), (lon_max, lat_min), (lon_max, lat_max), (lon_min, lat_max), (lon_min, lat_min)])




# Apply function to get polygons
#df = pd.DataFrame(data['qdgccode'].unique())



#ata['geometry'] = data['qdgccode'].apply(qdgc_to_polygon)
data['geometry'] = data['qdgccode'].apply(qdgc_to_polygon)
#geom = qdgc_to_polygon(df[0].values())


In [ ]:
# Function to convert meters to degrees for latitude and longitude
def meters_to_degrees(lat, meters):
    # 1 degree latitude is roughly 111.32 km (constant)
    deg_lat = meters / 111320

    # 1 degree longitude is 111.32 km * cos(latitude) (varies with latitude)
    deg_lon = meters / (111320 * math.cos(math.radians(lat)))

    return deg_lat, deg_lon

# Function to convert MGRS to polygon
def mgrs_to_polygon(mgrs_code):
    mgrs_converter = mgrs.MGRS()

    # Get lower-left corner of MGRS grid square (lat, lon)
    lat, lon = mgrs_converter.toLatLon(mgrs_code)

    # Determine grid size in meters based on MGRS precision
    # Example: Adjust according to precision (1000m for 4-character code, etc.)
    grid_size_meters = 10000  # Adjust based on precision of MGRS code

    # Convert meters to degrees at the given latitude
    grid_size_lat_deg, grid_size_lon_deg = meters_to_degrees(lat, grid_size_meters)

    # Create polygon points for the grid square
    polygon_points = [
        (lon, lat),  # lower-left
        (lon + grid_size_lon_deg, lat),  # lower-right
        (lon + grid_size_lon_deg, lat + grid_size_lat_deg),  # upper-right
        (lon, lat + grid_size_lat_deg)  # upper-left
    ]

    # Create the polygon using shapely
    polygon = Polygon(polygon_points)

    return polygon

m = mgrs.MGRS()
# Function to convert MGRS to UTM polygon
def mgrs_to_utm_polygon(mgrs_code):
    # Convert MGRS to lat/lon using the mgrs library

    lat, lon = m.toLatLon(mgrs_code)  # Get lower-left corner in lat/lon

    # Extract UTM zone number from the MGRS code (first two digits are the UTM zone)
    utm_zone_number = int(mgrs_code[:2])

    # Determine if it's in the northern or southern hemisphere based on the latitude band
    hemisphere = 'north' if mgrs_code[2].upper() >= 'N' else 'south'

    # Create UTM projection based on the zone number and hemisphere
    utm_proj = Proj(proj='utm', zone=utm_zone_number, ellps='WGS84', south=(hemisphere == 'south'))

    # Transformer to convert lat/lon to UTM coordinates (EPSG:4326 -> UTM)
    transformer_to_utm = Transformer.from_crs("epsg:4326", utm_proj.srs)

    # Transform the lower-left corner from lat/lon to UTM (meters)
    x_utm, y_utm = transformer_to_utm.transform(lat, lon)

    # Define the grid size in meters (e.g., 1000 meters for a 1 km MGRS grid)
    grid_size_meters = 10000  # Adjust based on the precision of your MGRS code

    # Create the UTM polygon points (lower-left, lower-right, upper-right, upper-left)
    utm_polygon_points = [
        (x_utm, y_utm),                                 # lower-left
        (x_utm + grid_size_meters, y_utm),              # lower-right
        (x_utm + grid_size_meters, y_utm + grid_size_meters),  # upper-right
        (x_utm, y_utm + grid_size_meters)               # upper-left
    ]

    # Create the polygon in UTM space
    utm_polygon = Polygon(utm_polygon_points)

    # Transformer to convert UTM coordinates back to lat/lon (UTM -> EPSG:4326)
    transformer_to_latlon = Transformer.from_crs(utm_proj.srs, "epsg:4326")

    # Transform the UTM polygon back to lat/lon coordinates
    latlon_polygon = Polygon([transformer_to_latlon.transform(x, y) for x, y in utm_polygon.exterior.coords])

    return latlon_polygon

#data['geometry'] = data['mgrscode'].apply(mgrs_to_polygon)